Run these lines once:

In [1]:
import sys
!conda install --yes --prefix {sys.prefix} numpy
!conda install --yes --prefix {sys.prefix} pandas
!conda install --yes --prefix {sys.prefix} scipy

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /Users/lavanyasingh/anaconda3

  added / updated specs:
    - scipy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scipy-1.3.2                |   py36h1410ff5_0        12.5 MB
    ------------------------------------------------------------
                                           Total:        12.5 MB

The following NEW packages will be INSTALLED:

  scipy              pkgs/main/osx-64::scipy-1.3.2-py36h1410ff5_0



scipy-1.3.2          | 12.5 MB   | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


Read data from csv into pandas dataframe

In [1]:
import pandas as pd
import csv

data = {"source":[], "dest_ip":[], "dest_owner":[], "type":[], "OS": [], "length":[], "app":[], "category":[]}
with open("parsed_2.csv", "r") as inf:
    reader = csv.reader(inf, delimiter=',', quotechar = '"')
    next(reader)
    for row in reader:
        data["source"].append(row[0])
        data["dest_ip"].append(row[1])
        data["dest_owner"].append(row[2][0:row[2].find(" ")].strip())
        data["type"].append(row[3])
        data["OS"].append(row[4])
        data["length"].append(row[5])
        data["app"].append(row[6])
        data["category"].append(row[7])
df = pd.DataFrame(data)
print(df.head(15))


         source         dest_ip                dest_owner type   OS length  \
0   192.168.2.4   17.120.254.12        APPLE-ENGINEERING,  tcp  ios     71   
1   192.168.2.4   17.120.254.12        APPLE-ENGINEERING,  tcp  ios     40   
2   192.168.2.4   17.120.254.12        APPLE-ENGINEERING,  tcp  ios     40   
3   192.168.2.4  17.248.128.144        APPLE-ENGINEERING,  tcp  ios     83   
4   192.168.2.4  17.248.128.143        APPLE-ENGINEERING,  tcp  ios     83   
5   192.168.2.4  17.248.128.173        APPLE-ENGINEERING,  tcp  ios     83   
6   192.168.2.4  17.248.128.144        APPLE-ENGINEERING,  tcp  ios     52   
7   192.168.2.4  17.248.128.143        APPLE-ENGINEERING,  tcp  ios     52   
8   192.168.2.4  17.248.128.173        APPLE-ENGINEERING,  tcp  ios     52   
9   192.168.2.4  17.248.128.144        APPLE-ENGINEERING,  tcp  ios     52   
10  192.168.2.4  17.248.128.173        APPLE-ENGINEERING,  tcp  ios     52   
11  192.168.2.4  17.248.128.143        APPLE-ENGINEERING,  tcp  

Top 15 most frequent providers (based on packet count)

In [2]:
top_fifteen = df[df.dest_owner != ""]["dest_owner"].value_counts()[0:15]
print(top_fifteen)

AMAZON-02,                      64856
AKAMAI-AS,                      42328
GOOGLE,                         22274
AKAMAI-ASN1,                    15352
HIGHWINDS3,                      9759
CMCS,                            8538
CLOUDFLARENET,                   6397
AMAZON-AES,                      5911
EDGECAST,                        4628
FASTLY,                          4185
FACEBOOK,                        3933
LEVEL3,                          3737
APPLE-ENGINEERING,               2113
MOPUB,                            916
MICROSOFT-CORP-MSN-AS-BLOCK,      908
Name: dest_owner, dtype: int64


Previous analysis does not take into account the differences in number of packets for each app (this is why roblox is so high). 
Let's rank providers based on the proportion of packets captured for each app. 


Score for a given provider is given by  $$Score = \frac{1}{N} \sum_{n=1}^{N} \frac{P_n}{T_n}$$

Where $N$ is the number of apps, $P_n$ is the number of outgoing packets for app $n$ handled by the provider, and $T_n$ is the total number of outgoing packets for app $n$. Thus, the maximum possible score (all packets handled for all apps) is 1.

In [3]:
def get_scores(df):
    owner_dict = {key:0 for key in df.dest_owner.unique()}
    df = df[df.dest_owner != ""]
    
    
    for app in df.app.unique():
        perc = df[df.app == app]["dest_owner"].value_counts()
        total = len(df[df.app == app])
        perc = {key: float(value/total) for key, value in perc.items()}
        for key,value in perc.items(): owner_dict[key] += value
        
    apps = len(df.app.unique())
    score = {key: float(value)/apps for key,value in owner_dict.items()}
    scores = pd.DataFrame(sorted(score.items(), key=lambda x: x[1], reverse=True), columns = ["Provider", "Score"])
    scores["% Traffic"] = scores["Provider"].map(lambda x: len(df[df.dest_owner == x])/len(df)*100)
    return scores

print(get_scores(df).head(15))

              Provider     Score  % Traffic
0           AMAZON-02,  0.228789  32.430594
1              GOOGLE,  0.183949  11.137891
2           AKAMAI-AS,  0.106181  21.165693
3            FACEBOOK,  0.072727   1.966657
4          AMAZON-AES,  0.068440   2.955736
5   APPLE-ENGINEERING,  0.063912   1.056585
6                CMCS,  0.047298   4.269342
7       CLOUDFLARENET,  0.041984   3.198756
8          HIGHWINDS3,  0.040244   4.879890
9              FASTLY,  0.035119   2.092667
10        AKAMAI-ASN1,  0.032611   7.676614
11           EDGECAST,  0.018572   2.314185
12            BADOO-U,  0.011181   0.133011
13             PAYPAL,  0.010092   0.434035
14             LEVEL3,  0.008187   1.868649


Let's repeat the same thing for ios.

In [4]:
print(get_scores(df[df.OS == "ios"]).head(15))

              Provider     Score  % Traffic
0           AMAZON-02,  0.228789  32.430594
1              GOOGLE,  0.183949  11.137891
2           AKAMAI-AS,  0.106181  21.165693
3            FACEBOOK,  0.072727   1.966657
4          AMAZON-AES,  0.068440   2.955736
5   APPLE-ENGINEERING,  0.063912   1.056585
6                CMCS,  0.047298   4.269342
7       CLOUDFLARENET,  0.041984   3.198756
8          HIGHWINDS3,  0.040244   4.879890
9              FASTLY,  0.035119   2.092667
10        AKAMAI-ASN1,  0.032611   7.676614
11           EDGECAST,  0.018572   2.314185
12            BADOO-U,  0.011181   0.133011
13             PAYPAL,  0.010092   0.434035
14             LEVEL3,  0.008187   1.868649


And for android.

In [5]:
print(get_scores(df[df.OS == "android"]).head(15))

Empty DataFrame
Columns: [Provider, Score, % Traffic]
Index: []


android talks to google WAY more -> indicative of background/non-app related traffic???

Now let's analyze the distribution of packet providers. Let's calculate Shannon's entropy for each app. This value will be highest if providers are evenly distributed. 

In [6]:
from scipy.stats import entropy

app_dict = {app:None for app in df.app.unique()}
for app in df.app.unique():
    counts = [len(df[df["app"] == app][df["dest_owner"] == dest_owner]) for dest_owner in df.dest_owner.unique()]
    app_dict[app] = entropy(counts, base=None)
app_entropy = pd.DataFrame(sorted(app_dict.items(), key=lambda x: x[1], reverse=True), columns = ['App', 'Entropy'])
app_entropy["CDN Count"] = [len(df[df.app == app].dest_owner.unique()) for app in app_entropy['App']]
app_entropy["Top CDN"] = [df[df.app == app]['dest_owner'].value_counts().idxmax() for app in app_entropy['App']]
print(app_entropy)

/Users/lavanyasingh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


              App   Entropy  CDN Count             Top CDN
0        woodshop  2.159689         18         HIGHWINDS3,
1   watermarbling  2.125502         17         HIGHWINDS3,
2        epicrace  2.117777         18          AMAZON-02,
3        textfree  2.068500         14          AMAZON-02,
4     woodturning  1.938164         20           EDGECAST,
..            ...       ...        ...                 ...
56     googlehome  0.258126          3             GOOGLE,
57         noggin  0.167389         11          AKAMAI-AS,
58      lingokids  0.104512          8          AMAZON-02,
59    youtubekids  0.067905          3             GOOGLE,
60       baseline  0.000000          1  APPLE-ENGINEERING,

[61 rows x 4 columns]


Now let's do some entropy analysis by category. I'm going to manually input categories :(

In [48]:
categories = {"forgeahead":"games", "tinder":"lifestyle", "luckygo":"lifestyle", "arrive":"shopping", "yolo":"social", 
              "venmo":"finance", "walmart":"shopping", "cashapp":"finance", "tiktok":"social", "messenger":"social", 
             "amazonshopping":"shopping", "roblox":"games", "googlehome":"lifestyle", "amazon":"shopping", "disneyplus":"entertainment", 
             "messengerkids":"kids", "baseline":"control", "snapchat":"social", "youtubekids":"kids", "androidauto":"auto"}
app_entropy["category"] = [categories[app] for app in app_entropy.App.unique()]
# making the provider names shorter lol
app_entropy["Top CDN"] = [provider[0:15] for provider in app_entropy["Top CDN"]]
print(app_entropy)

               App   Entropy  CDN Count       Top CDN       category
0       forgeahead  2.223228         24     AKAMAI-AS          games
1           tinder  1.914604         10        GOOGLE      lifestyle
2          luckygo  1.560211         13     AKAMAI-AS      lifestyle
3           arrive  1.425340          7        GOOGLE       shopping
4             yolo  1.368666         11        GOOGLE         social
5            venmo  1.367446         13     AMAZON-02        finance
6          walmart  1.158229          9          CMCS       shopping
7          cashapp  1.127729          8        GOOGLE        finance
8           tiktok  1.021691          9  AKAMAI-ASN1,         social
9        messenger  0.950322         11        GOOGLE         social
10  amazonshopping  0.863552          5          CMCS       shopping
11          roblox  0.750157          9          CMCS          games
12      googlehome  0.697491          4        GOOGLE      lifestyle
13          amazon  0.694052      

Now let's get entropy by category.

In [49]:
# a little maintenance
df["category"] = [categories[app] for app in df.app]
df["dest_owner"] = [provider[0:15] for provider in df.dest_owner]

cat_dict = {category:None for category in categories.values()}
for category in df.category.unique():
    counts = [len(df[df["category"] == category][df["dest_owner"] == dest_owner]) for dest_owner in df.dest_owner.unique()]
    counts = list(filter(lambda x: x!= 0, counts))
    cat_dict[category] = entropy(counts, base=None)
cat_entropy = pd.DataFrame(cat_dict.items(), columns = ['Category', 'Entropy'])
cat_entropy.sort_values(by=["Entropy"])
cat_entropy["CDN Count"] = [len(df[df.category == category].dest_owner.unique()) for category in cat_entropy['Category']]
cat_entropy["Top CDN"] = [df[df.category == category]['dest_owner'].value_counts().idxmax() for category in cat_entropy['Category']]
cat_entropy["# of Apps"] = [len(df[df.category == category].app.unique()) for category in cat_entropy['Category']]
print(cat_entropy)
    

/Users/lavanyasingh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


        Category   Entropy  CDN Count    Top CDN  # of Apps
0          games  1.542955         26       CMCS          2
1      lifestyle  1.914956         15  AKAMAI-AS          3
2       shopping  1.015942         12       CMCS          4
3         social  1.191922         18     GOOGLE          4
4        finance  1.594803         14  AMAZON-02          2
5  entertainment  0.677383          8  AMAZON-02          1
6           kids  0.172813          4     GOOGLE          2
7        control  0.339136          3     GOOGLE          1
8           auto  0.007168          2     GOOGLE          1


Keep in mind that the sample size for each category is pretty tiny.

Let's now try to figure out how distributed the entire dataset is. I'm going to use a a chi squared to test to figure out 
how evenly distributed CDN usage is.

In [50]:
from scipy.stats import chisquare
stat = chisquare([len(df[df["dest_owner"] == provider]) for provider in df.dest_owner.unique()])
print(stat.pvalue)

0.0


Our pvalue is 0. I guess CDN usage is nowhere NEAR unformly distributed. 

It also seems like "smaller" apps are more likely to use more CDN's than an app owned by a huge company, like Google. To test this, I'm going to use the app's downloads (sum of ios and android downloads as given by sensortower) as a rough proxy measure for app size (but maybe we can come up with something better). I'm going to have to manually input this :(

In [51]:
downloads = {"forgeahead":10000000, "tinder":6000000, "luckygo":1000000, "arrive":1800000, "yolo":2600000, 
              "venmo":1700000, "walmart":2000000, "cashapp":3000000, "tiktok":58000000, "messenger":43000000, 
             "amazonshopping":9000000, "roblox":12000000, "googlehome":7000000, "amazon":9000000, "disneyplus":8000000, 
             "messengerkids":900000, "baseline":0, "snapchat":26000000, "youtubekids":7000000, "androidauto":3000000}
df["downloads"] = [downloads[app] for app in df.app]
app_entropy["downloads"] = [downloads[app] for app in app_entropy.App]
app_entropy.sort_values(by=["downloads"])
print(app_entropy)


               App   Entropy  CDN Count       Top CDN       category  \
0       forgeahead  2.223228         24     AKAMAI-AS          games   
1           tinder  1.914604         10        GOOGLE      lifestyle   
2          luckygo  1.560211         13     AKAMAI-AS      lifestyle   
3           arrive  1.425340          7        GOOGLE       shopping   
4             yolo  1.368666         11        GOOGLE         social   
5            venmo  1.367446         13     AMAZON-02        finance   
6          walmart  1.158229          9          CMCS       shopping   
7          cashapp  1.127729          8        GOOGLE        finance   
8           tiktok  1.021691          9  AKAMAI-ASN1,         social   
9        messenger  0.950322         11        GOOGLE         social   
10  amazonshopping  0.863552          5          CMCS       shopping   
11          roblox  0.750157          9          CMCS          games   
12      googlehome  0.697491          4        GOOGLE      lifes

Conclusions: There is definitely a lot of clustering going on, and sampling by category seems to be worth it because there's a good amount of variance in the entropy of each category. I think for future data collection we should also collect information about the size of the company that owns the app, the category of the app (so I don't have to input it manually lol), and the country of the app's owner (I didn't analyze that here, but TikTok talks to a Chinese CDN whereas the rest talk to American ones and that might be interesting).


How many CDN's do apps tend to use? The more CDN's, the more chances for failure.

In [52]:
counts = {}
for app in df.app.unique():
    counts.update({app:len(df[df["app"] == app].dest_owner.unique())})
counts_df = pd.DataFrame(counts.items(), columns=["app", "# of CDN's"])    
counts_df = counts_df.sort_values(by="# of CDN's", ascending=False)
counts_df["category"] = [categories[app] for app in counts_df["app"]]
print(counts_df)

               app  # of CDN's       category
8       forgeahead          24          games
10           venmo          13        finance
13         luckygo          13      lifestyle
16       messenger          11         social
9             yolo          11         social
6           tinder          10      lifestyle
11          tiktok           9         social
18          roblox           9          games
15         walmart           9       shopping
14          amazon           8       shopping
0       disneyplus           8  entertainment
4          cashapp           8        finance
12          arrive           7       shopping
7   amazonshopping           5       shopping
1       googlehome           4      lifestyle
5         snapchat           4         social
17   messengerkids           4           kids
2         baseline           3        control
19     youtubekids           3           kids
3      androidauto           2           auto


I'm now manually going to tag certain CDN's as belonging to certain apps so that we can analyze 3p CDN usage.

In [60]:
print(list(df.dest_owner.unique()))

['AKAMAI-AS', '', 'AMAZON-02', 'APPLE-ENGINEERI', 'GOOGLE', 'FASTLY', 'CMCS', 'SQUARE', 'AMAZON-AES', 'HIGHWINDS3', 'FACEBOOK', 'HIGHWINDS2', 'EDGECAST', 'TAOBAO', 'CNNIC-ALIBABA-U', 'CLOUDFLARENET', 'AS-CHOOPA', 'MICROSOFT-CORP-', 'AS-CRITEO', 'COGECO-PEER1', 'DOUBLE-VERIFY', 'SOFTLAYER', 'SECTIGO,', 'APPLE-AUSTIN', 'BRAINTREEPAYMEN', 'PAYPAL', 'OVH,', 'AKAMAI-ASN1,', 'MONTICELLO', 'ASN-APPNEXUS', 'XMISSION', 'MOPUB', 'TENCENT-NET-AP-', 'BJ-GUANGHUAN-AP', 'AS-26496-GO-DAD', 'TWITTER', 'EBAY', 'ROBLOX-PRODUCTI', 'LLNW']


In [63]:
first_party = {"venmo":"PAYPAL", "venmo":"BRAINTREEPAYMEN", "messenger":"FACEBOOK", "roblox":"ROBLOX-PRODUCTI", 
               "amazon":"AMAZON-AES", "amazon":"AMAZON-02", "cashapp":"SQUARE", "amazonshopping":"AMAZON-AES", 
              "amazonshopping":"AMAZON-02", "googlehome":"GOOGLE", "messengerkids":"FACEBOOK", "youtubekids":"GOOGLE", 
              "androidauto":"GOOGLE"}
self_cdn = {app:False for app in df.dest_owner.unique()}
for app in self_cdn:
    if 

print(df.head)

SyntaxError: invalid syntax (<ipython-input-63-2e8c9e76c46c>, line 7)